In [21]:
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')

from IPython.display import display

try:
    import cvxpy as cp
    USE_CVXPY = True
except ImportError:
    USE_CVXPY = False
    print("cvxpy not installed, optimization will be skipped")

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

ROOT = Path().resolve().parent
DATA_PATH = ROOT / "data" / "raw"
TABLE_PATH = ROOT / "reports" / "tables"
FIGURE_PATH = ROOT / "reports" / "figures"
TABLE_PATH.mkdir(parents=True, exist_ok=True)
FIGURE_PATH.mkdir(parents=True, exist_ok=True)


def set_academic_style():
    mpl.rcParams.update({
        "figure.figsize": (10, 4),
        "axes.facecolor": "white",
        "savefig.facecolor": "white",
        "axes.grid": True,
        "grid.color": "#ECEFF3",
        "grid.linestyle": "-",
        "grid.linewidth": 0.6,
        "font.size": 11,
        "axes.titlesize": 12,
        "axes.labelsize": 11,
        "legend.fontsize": 10,
        "xtick.labelsize": 10,
        "ytick.labelsize": 10,
    })


def save_figure(fig, filename):
    fig.tight_layout()
    output_path = FIGURE_PATH / filename
    fig.savefig(output_path, dpi=300, bbox_inches="tight")
    plt.show()


def display_table(df, filename):
    output_path = TABLE_PATH / filename
    df.to_csv(output_path, index=False)
    display(df)


set_academic_style()


In [22]:
optim_file = DATA_PATH / "optimisation.csv"
if optim_file.exists():
    df_raw = pd.read_csv(optim_file, parse_dates=["timestamp"])
    print(f"Loaded optimisation.csv with {len(df_raw)} rows")
    print(f"Available columns: {df_raw.columns.tolist()}")
    
    df = df_raw.head(24).copy()
    
    if "Demand" not in df.columns:
        hours = df["timestamp"].dt.hour
        df["Demand"] = 2.0 + 1.5 * np.sin((hours - 6) * np.pi / 12) + np.random.normal(0, 0.1, len(df))
        print("Generated synthetic demand pattern")
    
    if "PV" not in df.columns:
        if "pv_low" in df.columns and "pv_high" in df.columns:
            df["PV"] = (df["pv_low"] + df["pv_high"]) / 2
        else:
            hours = df["timestamp"].dt.hour
            df["PV"] = np.maximum(0, 3.0 * np.sin((hours - 6) * np.pi / 12))
        print("Generated PV from available data")
    
    if "Price_buy" not in df.columns:
        if "Price" in df.columns:
            df["Price_buy"] = df["Price"] * 1.2
            df["Price_sell"] = df["Price"] * 0.6
        else:
            hours = df["timestamp"].dt.hour
            df["Price_buy"] = 0.25 + 0.10 * (hours >= 17) + 0.05 * (hours >= 9) * (hours < 17)
            df["Price_sell"] = 0.08 + 0.04 * (hours >= 10) * (hours < 16)
        print("Generated price data from available data")
        
else:
    print("optimisation.csv not found, generating synthetic data")
    dates = pd.date_range("2024-07-01", periods=24, freq="h")
    hours = dates.hour
    demand_pattern = 2.0 + 1.5 * np.sin((hours - 6) * np.pi / 12)
    pv_pattern = np.maximum(0, 3.0 * np.sin((hours - 6) * np.pi / 12))
    
    df = pd.DataFrame({
        "timestamp": dates,
        "Demand": demand_pattern + np.random.normal(0, 0.1, 24),
        "PV": pv_pattern * (0.8 + np.random.uniform(0, 0.4, 24)),
        "Price_buy": 0.25 + 0.10 * (hours >= 17) + 0.05 * (hours >= 9) * (hours < 17),
        "Price_sell": 0.08 + 0.04 * (hours >= 10) * (hours < 16)
    })

df = df.sort_values("timestamp").reset_index(drop=True)
assert df["timestamp"].is_monotonic_increasing, "Timestamps must be strictly increasing"

num_cols = df.select_dtypes(include=[np.number]).columns
if df[num_cols].isna().any().any():
    df[num_cols] = df[num_cols].interpolate(limit_direction="both").fillna(method="ffill").fillna(method="bfill")
if df[num_cols].isna().any().any():
    raise ValueError("Numeric columns contain NaNs after filling")

print(f"\nOptimization horizon: {len(df)} hours")
print(f"Demand range: {df['Demand'].min():.2f} - {df['Demand'].max():.2f} kW")
print(f"PV range: {df['PV'].min():.2f} - {df['PV'].max():.2f} kW")
print(f"Price buy range: €{df['Price_buy'].min():.3f} - €{df['Price_buy'].max():.3f}/kWh")
print(f"Price sell range: €{df['Price_sell'].min():.3f} - €{df['Price_sell'].max():.3f}/kWh")

Loaded optimisation.csv with 24 rows
Available columns: ['timestamp', 'pv_low', 'pv_high', 'Price', 'Temperature', 'Pressure (hPa)', 'Cloud_cover (%)', 'Cloud_cover_low (%)', 'Cloud_cover_mid (%)', 'Cloud_cover_high (%)', 'Wind_speed_10m (km/h)', 'Shortwave_radiation (W/m²)', 'direct_radiation (W/m²)', 'diffuse_radiation (W/m²)', 'direct_normal_irradiance (W/m²)']
Generated synthetic demand pattern
Generated PV from available data
Generated price data from available data

Optimization horizon: 24 hours
Demand range: 0.41 - 3.52 kW
PV range: 0.00 - 1.43 kW
Price buy range: €0.060 - €0.123/kWh
Price sell range: €0.030 - €0.061/kWh


In [23]:
PV_CAP = 5.0
BATT_CAP = 10.0
BATT_POWER = 5.0
GRID_LIMIT = 5.0
EFFICIENCY = 0.95
T = 24

print(f"System parameters: PV={PV_CAP}kW, Battery={BATT_CAP}kWh, Power={BATT_POWER}kW, Grid={GRID_LIMIT}kW")

System parameters: PV=5.0kW, Battery=10.0kWh, Power=5.0kW, Grid=5.0kW


In [24]:
def optimize_storage(demand, pv, price_buy, price_sell):
    """
    Optimize battery storage control for a household energy management system.
    
    Objective: Minimize total cost = sum(price_buy * grid_import - price_sell * grid_export)
    
    Decision variables:
    - soc[t]: State of charge (battery energy level) at time t
    - charge[t]: Battery charging power at time t
    - discharge[t]: Battery discharging power at time t
    - grid_import[t]: Power imported from grid at time t
    - grid_export[t]: Power exported to grid at time t
    
    Constraints:
    - Energy balance: PV + grid_import + battery_discharge = demand + grid_export + battery_charge
    - Battery dynamics: soc[t+1] = soc[t] + efficiency * charge[t] - discharge[t] / efficiency
    - Battery limits: 0 <= soc[t] <= BATT_CAP
    - Power limits: charge/discharge <= BATT_POWER, grid <= GRID_LIMIT
    """
    if not USE_CVXPY:
        print("ERROR: cvxpy not available. Install with: pip install cvxpy")
        return None
    
    T = len(demand)
    
    soc = cp.Variable(T+1)
    charge = cp.Variable(T)
    discharge = cp.Variable(T)
    grid_import = cp.Variable(T)
    grid_export = cp.Variable(T)
    
    cost = cp.sum(cp.multiply(price_buy, grid_import) - cp.multiply(price_sell, grid_export))
    
    constraints = [
        soc[0] == BATT_CAP * 0.5,
        soc[T] >= BATT_CAP * 0.2,
    ]
    
    for t in range(T):
        constraints += [
            soc[t+1] == soc[t] + EFFICIENCY * charge[t] - discharge[t] / EFFICIENCY,
            soc[t+1] >= 0,
            soc[t+1] <= BATT_CAP,
            charge[t] >= 0,
            charge[t] <= BATT_POWER,
            discharge[t] >= 0,
            discharge[t] <= BATT_POWER,
            grid_import[t] >= 0,
            grid_import[t] <= GRID_LIMIT,
            grid_export[t] >= 0,
            grid_export[t] <= GRID_LIMIT,
            pv[t] + grid_import[t] + discharge[t] == demand[t] + grid_export[t] + charge[t]
        ]
    
    problem = cp.Problem(cp.Minimize(cost), constraints)
    
    try:
        problem.solve(solver=cp.ECOS, verbose=False)
    except:
        try:
            problem.solve(solver=cp.SCS, verbose=False)
        except:
            print("All solvers failed")
            return None
    
    if problem.status not in ["optimal", "optimal_inaccurate"]:
        print(f"Optimization failed: {problem.status}")
        return None
    
    return {
        "soc": soc.value,
        "charge": charge.value,
        "discharge": discharge.value,
        "grid_import": grid_import.value,
        "grid_export": grid_export.value,
        "cost": problem.value,
        "status": problem.status
    }

In [25]:
demand = df["Demand"].values
pv_base = df["PV"].values
price_buy = df["Price_buy"].values
price_sell = df["Price_sell"].values

pv_low = pv_base * 0.5
pv_high = pv_base * 1.5

print("=== Scenario 1: PV_low (50% of baseline) ===")
result_low = optimize_storage(demand, pv_low, price_buy, price_sell)
if result_low:
    assert result_low["status"] in {"optimal", "optimal_inaccurate"}, "PV_low solver did not converge"
    if not np.all((result_low["soc"] >= -1e-6) & (result_low["soc"] <= BATT_CAP + 1e-6)):
        raise ValueError("PV_low SOC out of bounds")

print("\n=== Scenario 2: PV_high (150% of baseline) ===")
result_high = optimize_storage(demand, pv_high, price_buy, price_sell)
if result_high:
    assert result_high["status"] in {"optimal", "optimal_inaccurate"}, "PV_high solver did not converge"
    if not np.all((result_high["soc"] >= -1e-6) & (result_high["soc"] <= BATT_CAP + 1e-6)):
        raise ValueError("PV_high SOC out of bounds")

if result_low:
    print(f"\nPV_low optimization cost: €{result_low['cost']:.2f}")
    print(f"Status: {result_low['status']}")
if result_high:
    print(f"\nPV_high optimization cost: €{result_high['cost']:.2f}")
    print(f"Status: {result_high['status']}")

=== Scenario 1: PV_low (50% of baseline) ===

=== Scenario 2: PV_high (150% of baseline) ===

PV_low optimization cost: €3.90
Status: optimal

PV_high optimization cost: €3.17
Status: optimal

=== Scenario 2: PV_high (150% of baseline) ===

PV_low optimization cost: €3.90
Status: optimal

PV_high optimization cost: €3.17
Status: optimal


In [26]:
if result_low and result_high:
    summary_data = []
    
    for name, result, pv in [("PV_low", result_low, pv_low), ("PV_high", result_high, pv_high)]:
        total_cost = result["cost"]
        energy_bought = np.sum(result["grid_import"])
        energy_sold = np.sum(result["grid_export"])
        battery_throughput = np.sum(result["discharge"])
        battery_cycles = battery_throughput / BATT_CAP
        max_soc = np.max(result["soc"])
        min_soc = np.min(result["soc"])
        
        summary_data.append({
            "Scenario": name,
            "Total_cost_EUR": round(total_cost, 2),
            "Energy_bought_kWh": round(energy_bought, 2),
            "Energy_sold_kWh": round(energy_sold, 2),
            "Battery_cycles": round(battery_cycles, 2),
            "SOC_max_kWh": round(max_soc, 2),
            "SOC_min_kWh": round(min_soc, 2)
        })
    
    summary_df = pd.DataFrame(summary_data)
    print("\n=== Optimization Summary ===")
    print(summary_df.to_string(index=False))
    display_table(summary_df, "11_storage_optimization_summary.csv")
else:
    print("Optimization failed, cannot generate summary")



=== Optimization Summary ===
Scenario  Total_cost_EUR  Energy_bought_kWh  Energy_sold_kWh  Battery_cycles  SOC_max_kWh  SOC_min_kWh
  PV_low            3.90              42.27              0.0            1.23         10.0          0.0
 PV_high            3.17              35.15              0.0            1.23         10.0          0.0


,Scenario,Total_cost_EUR,Energy_bought_kWh,Energy_sold_kWh,Battery_cycles,SOC_max_kWh,SOC_min_kWh
0,PV_low,3.90,42.27,0.0,1.23,10.0,0.0
1,PV_high,3.17,35.15,0.0,1.23,10.0,0.0


In [27]:
if result_low and result_high:
    hours = np.arange(T)
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    axes[0, 0].plot(hours, demand, label="Demand", color="black", linewidth=2)
    axes[0, 0].plot(hours, pv_low, label="PV_low", color="orange", linestyle="--")
    axes[0, 0].set_ylabel("Power (kW)")
    axes[0, 0].set_title("PV_low: Demand & PV")
    axes[0, 0].legend()
    axes[0, 0].grid(alpha=0.3)
    
    axes[0, 1].plot(hours, result_low["grid_import"], label="Import", color="red")
    axes[0, 1].plot(hours, result_low["grid_export"], label="Export", color="green")
    axes[0, 1].set_ylabel("Power (kW)")
    axes[0, 1].set_title("PV_low: Grid flows")
    axes[0, 1].legend()
    axes[0, 1].grid(alpha=0.3)
    
    axes[1, 0].plot(hours, result_low["soc"][:-1], label="SOC", color="blue", linewidth=2)
    axes[1, 0].axhline(BATT_CAP, color="gray", linestyle="--", label="Max capacity")
    axes[1, 0].set_xlabel("Hour")
    axes[1, 0].set_ylabel("Energy (kWh)")
    axes[1, 0].set_title("PV_low: Battery SOC")
    axes[1, 0].legend()
    axes[1, 0].grid(alpha=0.3)
    
    axes[1, 1].plot(hours, result_low["charge"], label="Charge", color="green")
    axes[1, 1].plot(hours, result_low["discharge"], label="Discharge", color="red")
    axes[1, 1].set_xlabel("Hour")
    axes[1, 1].set_ylabel("Power (kW)")
    axes[1, 1].set_title("PV_low: Battery power")
    axes[1, 1].legend()
    axes[1, 1].grid(alpha=0.3)
    
    plt.tight_layout()
    save_figure(fig, "11_optimization_PV_low.png")
    print("PV_low optimization plot saved and displayed.")


PV_low optimization plot saved and displayed.


In [28]:
if result_low and result_high:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    axes[0, 0].plot(hours, demand, label="Demand", color="black", linewidth=2)
    axes[0, 0].plot(hours, pv_high, label="PV_high", color="orange", linestyle="--")
    axes[0, 0].set_ylabel("Power (kW)")
    axes[0, 0].set_title("PV_high: Demand & PV")
    axes[0, 0].legend()
    axes[0, 0].grid(alpha=0.3)
    
    axes[0, 1].plot(hours, result_high["grid_import"], label="Import", color="red")
    axes[0, 1].plot(hours, result_high["grid_export"], label="Export", color="green")
    axes[0, 1].set_ylabel("Power (kW)")
    axes[0, 1].set_title("PV_high: Grid flows")
    axes[0, 1].legend()
    axes[0, 1].grid(alpha=0.3)
    
    axes[1, 0].plot(hours, result_high["soc"][:-1], label="SOC", color="blue", linewidth=2)
    axes[1, 0].axhline(BATT_CAP, color="gray", linestyle="--", label="Max capacity")
    axes[1, 0].set_xlabel("Hour")
    axes[1, 0].set_ylabel("Energy (kWh)")
    axes[1, 0].set_title("PV_high: Battery SOC")
    axes[1, 0].legend()
    axes[1, 0].grid(alpha=0.3)
    
    axes[1, 1].plot(hours, result_high["charge"], label="Charge", color="green")
    axes[1, 1].plot(hours, result_high["discharge"], label="Discharge", color="red")
    axes[1, 1].set_xlabel("Hour")
    axes[1, 1].set_ylabel("Power (kW)")
    axes[1, 1].set_title("PV_high: Battery power")
    axes[1, 1].legend()
    axes[1, 1].grid(alpha=0.3)
    
    plt.tight_layout()
    save_figure(fig, "11_optimization_PV_high.png")
    print("PV_high optimization plot saved and displayed.")


PV_high optimization plot saved and displayed.


In [29]:
if result_low and result_high:
    battery_capacities = [5, 10, 15]
    sensitivity_results = []
    
    for batt_cap in battery_capacities:
        BATT_CAP_TEMP = batt_cap
        
        soc = cp.Variable(T+1)
        charge = cp.Variable(T)
        discharge = cp.Variable(T)
        grid_import = cp.Variable(T)
        grid_export = cp.Variable(T)
        
        cost = cp.sum(cp.multiply(price_buy, grid_import) - cp.multiply(price_sell, grid_export))
        
        constraints = [
            soc[0] == BATT_CAP_TEMP * 0.5,
            soc[T] >= BATT_CAP_TEMP * 0.3,
        ]
        
        for t in range(T):
            constraints += [
                soc[t+1] == soc[t] + EFFICIENCY * charge[t] - discharge[t] / EFFICIENCY,
                soc[t+1] >= 0,
                soc[t+1] <= BATT_CAP_TEMP,
                charge[t] >= 0,
                charge[t] <= BATT_POWER,
                discharge[t] >= 0,
                discharge[t] <= BATT_POWER,
                grid_import[t] >= 0,
                grid_import[t] <= GRID_LIMIT,
                grid_export[t] >= 0,
                grid_export[t] <= GRID_LIMIT,
                pv_base[t] + grid_import[t] + discharge[t] == demand[t] + grid_export[t] + charge[t]
            ]
        
        problem = cp.Problem(cp.Minimize(cost), constraints)
        
        try:
            problem.solve(solver=cp.ECOS, verbose=False)
        except:
            try:
                problem.solve(solver=cp.SCS, verbose=False)
            except:
                print(f"Solver failed for capacity {batt_cap}")
                continue
        
        if problem.status in ["optimal", "optimal_inaccurate"]:
            sensitivity_results.append({
                "Battery_capacity_kWh": batt_cap,
                "Total_cost": problem.value
            })
    
    if sensitivity_results:
        sensitivity_df = pd.DataFrame(sensitivity_results)
        print("\nSensitivity analysis:")
        print(sensitivity_df)
        display_table(sensitivity_df, "11_battery_sensitivity.csv")
        
        fig, ax = plt.subplots(figsize=(8, 5))
        ax.plot(sensitivity_df["Battery_capacity_kWh"], sensitivity_df["Total_cost"], 
                marker="o", linewidth=2, markersize=8)
        ax.set_xlabel("Battery capacity (kWh)")
        ax.set_ylabel("Total cost (€)")
        ax.set_title("Battery capacity sensitivity analysis")
        ax.grid(alpha=0.3)
        plt.tight_layout()
        save_figure(fig, "11_battery_sensitivity.png")
        print("Sensitivity analysis plot saved and displayed.")



Sensitivity analysis:
   Battery_capacity_kWh  Total_cost
0                     5    3.875341
1                    10    3.617922
2                    15    3.380534


,Battery_capacity_kWh,Total_cost
0,5,3.875341
1,10,3.617922
2,15,3.380534


Sensitivity analysis plot saved and displayed.


In [30]:
if result_low and result_high:
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    hours = np.arange(T)
    
    axes[0, 0].plot(hours, demand, 'k-', linewidth=2, label='Demand')
    axes[0, 0].plot(hours, pv_low, 'orange', linestyle='--', label='PV (50%)')
    axes[0, 0].fill_between(hours, 0, demand, alpha=0.1, color='black')
    axes[0, 0].fill_between(hours, 0, pv_low, alpha=0.2, color='orange')
    axes[0, 0].set_ylabel('Power (kW)')
    axes[0, 0].set_title('PV_low: Power flows')
    axes[0, 0].legend()
    axes[0, 0].grid(alpha=0.3)
    
    axes[0, 1].plot(hours, result_low["soc"][:-1], 'b-', linewidth=2, label='SOC')
    axes[0, 1].axhline(BATT_CAP, color='gray', linestyle='--', alpha=0.5, label='Max capacity')
    axes[0, 1].fill_between(hours, 0, result_low["soc"][:-1], alpha=0.2, color='blue')
    axes[0, 1].set_ylabel('Energy (kWh)')
    axes[0, 1].set_title('PV_low: Battery SOC')
    axes[0, 1].legend()
    axes[0, 1].grid(alpha=0.3)
    
    axes[0, 2].bar(hours, result_low["grid_import"], color='red', alpha=0.6, label='Import')
    axes[0, 2].bar(hours, -result_low["grid_export"], color='green', alpha=0.6, label='Export')
    axes[0, 2].axhline(0, color='black', linewidth=0.5)
    axes[0, 2].set_ylabel('Power (kW)')
    axes[0, 2].set_title('PV_low: Grid exchange')
    axes[0, 2].legend()
    axes[0, 2].grid(alpha=0.3)
    
    axes[1, 0].plot(hours, demand, 'k-', linewidth=2, label='Demand')
    axes[1, 0].plot(hours, pv_high, 'orange', linestyle='--', label='PV (150%)')
    axes[1, 0].fill_between(hours, 0, demand, alpha=0.1, color='black')
    axes[1, 0].fill_between(hours, 0, pv_high, alpha=0.2, color='orange')
    axes[1, 0].set_xlabel('Hour')
    axes[1, 0].set_ylabel('Power (kW)')
    axes[1, 0].set_title('PV_high: Power flows')
    axes[1, 0].legend()
    axes[1, 0].grid(alpha=0.3)
    
    axes[1, 1].plot(hours, result_high["soc"][:-1], 'b-', linewidth=2, label='SOC')
    axes[1, 1].axhline(BATT_CAP, color='gray', linestyle='--', alpha=0.5, label='Max capacity')
    axes[1, 1].fill_between(hours, 0, result_high["soc"][:-1], alpha=0.2, color='blue')
    axes[1, 1].set_xlabel('Hour')
    axes[1, 1].set_ylabel('Energy (kWh)')
    axes[1, 1].set_title('PV_high: Battery SOC')
    axes[1, 1].legend()
    axes[1, 1].grid(alpha=0.3)
    
    axes[1, 2].bar(hours, result_high["grid_import"], color='red', alpha=0.6, label='Import')
    axes[1, 2].bar(hours, -result_high["grid_export"], color='green', alpha=0.6, label='Export')
    axes[1, 2].axhline(0, color='black', linewidth=0.5)
    axes[1, 2].set_xlabel('Hour')
    axes[1, 2].set_ylabel('Power (kW)')
    axes[1, 2].set_title('PV_high: Grid exchange')
    axes[1, 2].legend()
    axes[1, 2].grid(alpha=0.3)
    
    plt.suptitle('Optimal Battery Control: PV_low vs PV_high Scenarios', fontsize=14, y=0.995)
    plt.tight_layout()
    save_figure(fig, "11_optimization_combined.png")
    print("Combined optimization plot saved and displayed.")


Combined optimization plot saved and displayed.


## Key Insights from Battery Optimization

**Scenario Comparison:**
- **PV_low (50%)**: Lower solar generation → More grid imports, higher costs
- **PV_high (150%)**: Higher solar generation → More self-consumption and grid exports, lower costs

**Optimization Strategy:**
1. **Peak shaving**: Battery discharges during high-demand/high-price periods
2. **Valley filling**: Battery charges during low-demand/low-price periods  
3. **PV utilization**: Excess PV is stored or exported based on price signals

**Battery Usage Pattern:**
- Charges when: PV > Demand AND price_sell < future price_buy
- Discharges when: Demand > PV AND price_buy is high
- Exports when: PV > Demand + Battery charge capacity

**Economic Impact:**
- Higher PV reduces grid dependency → Lower total cost
- Battery enables time-shifting of cheap/free solar energy
- Export revenue partially offsets import costs